# Cleaning of Dataset

In [14]:
import pandas as pd
import numpy as np
import os

# Define the file paths
input_file = 'C:\\Users\\Nidhi mishra\\OneDrive\\Desktop\\CashFlow_Forecasting_Raw_Dataset.xlsx'
output_file = 'C:\\Users\\Nidhi mishra\\OneDrive\\Desktop\\CashFlow_Forecasting_Cleaned_Dataset.xlsx'

# Ensure the output file is deleted if it exists and is not in use
try:
    if os.path.exists(output_file):
        os.remove(output_file)
except PermissionError:
    print(f"PermissionError: The file {output_file} is currently in use. Please close it and try again.")
    raise


Summaries saved to C:\Users\Nidhi mishra\OneDrive\Desktop\CashFlow_Forecasting_Summary.xlsx


# Save the Dataset

In [ ]:
# Function to clean and save dataset
def clean_and_save(sheet_name, output_sheet_name):
    # Load dataset
    df = pd.read_excel(input_file, sheet_name=sheet_name)

    # Fill missing values (example: numerical columns filled with mean)
    numeric_cols = df.select_dtypes(include=np.number).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

    # Fill missing values (example: categorical columns filled with mode)
    categorical_cols = df.select_dtypes(include='object').columns
    df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

    # Remove outliers (example using IQR method)
    Q1 = df[numeric_cols].quantile(0.25)
    Q3 = df[numeric_cols].quantile(0.75)
    IQR = Q3 - Q1

    df = df[~((df[numeric_cols] < (Q1 - 1.5 * IQR)) | (df[numeric_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

    # Save the cleaned dataset
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a' if os.path.exists(output_file) else 'w') as writer:
        df.to_excel(writer, sheet_name=output_sheet_name, index=False)
    
    return df

# Clean and save datasets, and capture the cleaned dataframes
cash_flow_cleaned = clean_and_save('Sheet1', 'Cash_Flow_Cleaned')
atm_maintenance_cleaned = clean_and_save('Predictive_Maintenance_Raw', 'ATM_Maintenance_Cleaned')
product_performance_cleaned = clean_and_save('Product_Performance_Raw', 'Product_Performance_Cleaned')
operational_efficiency_cleaned = clean_and_save('Operational_Metrics_Raw', 'Operational_Efficiency_Cleaned')



# Summarize the Dataset

In [ ]:
# Function to summarize data
def summarize_data(df):
    summary_numeric = df.select_dtypes(include=np.number).agg(['count', 'mean', 'median', 'std', lambda x: x.quantile(0.75) - x.quantile(0.25), 'nunique']).T
    summary_numeric.columns = ['Count', 'Mean', 'Median', 'Standard Deviation', 'IQR', 'Unique Values']
    summary_non_numeric = df.select_dtypes(exclude=np.number).agg(['count', 'nunique']).T
    summary_non_numeric.columns = ['Count', 'Unique Values']
    summary = pd.concat([summary_numeric, summary_non_numeric])
    summary['Missing Values'] = df.isnull().sum()
    return summary

# Summarize data
cash_flow_summary = summarize_data(cash_flow_cleaned)
predictive_maintenance_summary = summarize_data(atm_maintenance_cleaned)
product_performance_summary = summarize_data(product_performance_cleaned)
operational_metrics_summary = summarize_data(operational_efficiency_cleaned)


# Save Summary to file

In [ ]:
# Save summaries to a new Excel file
summary_file_path = r"C:\Users\Nidhi mishra\OneDrive\Desktop\CashFlow_Forecasting_Summary.xlsx"
with pd.ExcelWriter(summary_file_path, engine='openpyxl') as writer:
    cash_flow_summary.to_excel(writer, sheet_name='Cash_Flow_Summary')
    predictive_maintenance_summary.to_excel(writer, sheet_name='Predictive_Maintenance_Summary')
    product_performance_summary.to_excel(writer, sheet_name='Product_Performance_Summary')
    operational_metrics_summary.to_excel(writer, sheet_name='Operational_Metrics_Summary')

print(f"Summaries saved to {summary_file_path}")